# VK-бот

В целом VK-боты устроены по такому же принципу, что и боты в телеграме - они посылают запросы на сервер и когда поступает какой-то сигнал, они его получают и обрабатываются вашим кодом, например, можно ответить на сообщение или что-то сохранить. Вконтакте имеет два типа ботов, мы будет делать бота, который отвечает в лс от лица сообщества.


Конспект создан под влиянием двух постов с хабра ([вот](https://habr.com/ru/post/427691/) и [вот](https://habr.com/ru/post/428507/)), дока про сообщения ([вот](https://vk.com/dev/messages.send)) и документации пакетов для марковских цепец ([вот](https://github.com/jsvine/markovify)) и вк апи ([вот](https://github.com/python273/vk_api))

### Шаг 1. Создаем сообщество для бота

<img src="./pics/1_create_group.png"  style="width: 70%">
<img src="./pics/2_create_group_choose.png" style="width: 70%">
<img src="./pics/3_manage_groups.png" style="width: 70%">
<img src="./pics/4_new_group.png" style="width: 70%">

### Шаг 2. Получаем токен и настраиваем API


Заходим в раздел API

<img src="./pics/5_set_api.png"  style="width: 70%">


Включаем доступы


<img src="./pics/6_set_api_2.png" style="width: 70%">


Авторизуемся для токена - нужен номер телефона или приложение на телефоне.


<img src="./pics/7_auth.png" style="width: 70%">


Вот наш токен


<img src="./pics/8_token.png" style="width: 70%">


Включаем LongPoll API


<img src="./pics/9_logpoll.png" style="width: 70%">


Включаем доступы


<img src="./pics/10_set_access.png" style="width: 70%">

### Шаг 3. Устанавливаем пакет для работы с API

In [ ]:
! pip2 install vk_api

### Начинаем писать бота

(Pycharm). Заводим файл ```conf.py``` (или другое логичное название), где будут лежать данные про наше приложение, а именно айди группы (без минуса, просто число) и токен.

In [ ]:
from vk_api import VkUpload, VkApi
from vk_api.bot_longpoll import VkBotLongPoll, VkBotEventType
from conf import TOKEN, GROUP_ID

Сессия апи и авторизацией через токен, она используется другими сервисами

In [ ]:
vk_session = VkApi(token=TOKEN)

Сервис запросов, который проверяет, не пришло ли нам что-то

In [ ]:
long_poll = VkBotLongPoll(vk_session, GROUP_ID)

Еще один объект, который позволит вызывать разные методы из VK API. Например, отправлять сообщения. 

In [ ]:
vk = vk_session.get_api()

Загрузка объектов в вк. Понадобится для отправки картинок

In [ ]:
upload = VkUpload(vk_session)

Можно просто запустить такой цикл и он будет выполняться постоянн и ждать сообщений от сервера. Например, можно реагировать на отклик типа "новое сообщение" 

In [ ]:
for event in long_poll.listen():
    if event.type == VkBotEventType.MESSAGE_NEW:
        # DO SMTH

### Отвечаем на сообщения

Чтобы отправить ответ на сообщение пользователя, нужно несколько условий:

- достать айди пользователя
- сгенерировать уникальное число (требование такое, чтобы не повторялись запросы)
- вызвать нужный метод

In [1]:
import random

In [2]:
random.getrandbits(50)

191393640470370

In [ ]:
vk.messages.send(
    user_id=event.obj.from_id, 
    random_id=random.getrandbits(50),
    message="здесь могла быть ваша реклама"
)

In [ ]:
for event in long_poll.listen():
    if event.type == VkBotEventType.MESSAGE_NEW:
        vk.messages.send(
            user_id=event.obj.from_id, 
            random_id=random.getrandbits(50),
            message="я вас услышал"
        )

### Котики (и не только)

Скачиваем случайные изображения как на прошлой паре

In [ ]:
import requests

def get_dog_image():
    contents = requests.get('https://random.dog/woof.json').json()
    url = contents['url']
    image = requests.get(url, stream=True, timeout=2)
    return image

def get_cat_image():
    contents = requests.get('https://aws.random.cat/meow').json()
    url = contents['file']
    image = requests.get(url, stream=True, timeout=2)
    return image

Загружаем в вк, чтобы иметь возможность прислать кому-то. Возвращает конструкцию типа ```photo123124_121241241```, где первое число - айди владельца, а второе - айди картинки

In [ ]:
def upload_image(upload, image):
    photo = upload.photo_messages(photos=image.raw)[0]
    return f"photo{photo['owner_id']}_{photo['id']}"

In [ ]:
for event in long_poll.listen():
    if event.type == VkBotEventType.MESSAGE_NEW:
        image = get_cat_image()
        photo = upload_image(upload, image)
        vk.messages.send(
            user_id=event.obj.from_id, 
            random_id=random.getrandbits(50),
            message='держите кота', 
            attachment=photo
        )

### Толстой

Мы упоминали генерацию случайного текста с помощью марковской цепи, можно ее обучить и использовать вот так:

In [11]:
import markovify

In [3]:
corpus = open("/home/dkbrz/Downloads/voina-i-mir.txt").read()

In [14]:
text_model = markovify.Text(corpus, state_size=3)

Сохраняем

In [15]:
model_json = text_model.to_json()

In [16]:
model_json[:100]

'{"state_size": 3, "chain": "[[[\\"___BEGIN__\\", \\"___BEGIN__\\", \\"___BEGIN__\\"], {\\"\\\\u041b\\\\u0435\\\\u'

In [17]:
with open('mark_model.json', 'w') as f:
    f.write(model_json)

Считываем и подгружаем

In [18]:
model_json = open('mark_model.json').read()

In [20]:
model = markovify.Text.from_json(model_json)
model.make_short_sentence(280)

'Поговорив несколько времени в общем кругу, Сперанский встал и, подойдя к телеге, на которой была до приезда государя довольно сильная перестрелка, лежало несколько человек убитых и раненых, которые, казалось ему, ловят его за ноги.'

### Бот в целом

Хорошо бы все обернуть в функции и даже вынести в отдельный файл все вспомогательные. И обрабатывать ошибки. Первая - это таймаут на скачивание картинки, чтобы не ждать миллион лет. Вторая - для всего остального...

In [ ]:
def send_tolstoy(event):
    text = tolstoy.make_short_sentence(280)
    vk.messages.send(
        user_id=event.obj.from_id, 
        random_id=random.getrandbits(50),
        message=text
    )

def handle_new_message(event):
    if 'DOG' in event.obj.text:
        send_pet(event, kind='dog')
    elif 'CAT' in event.obj.text:
        send_pet(event, kind='cat')
    else:
        send_tolstoy(event)

def new_message_timeout_error(event):
    vk.messages.send(
        user_id=event.obj.from_id, 
        random_id=random.getrandbits(50),
        message="у меня интернет не очень и не успеваю качать картинки : ("
    )

def new_message_error(event):
    vk.messages.send(
        user_id=event.obj.from_id, 
        random_id=random.getrandbits(50),
        message="у меня что-то сломалось, давайте еще раз попробуем"
    )


def main():
    for event in long_poll.listen():
        if event.type == VkBotEventType.MESSAGE_NEW:
            try:
                handle_new_message(event)
            except TimeoutError:
                new_message_timeout_error(event)
            except:
                new_message_error(event)

if __name__ == '__main__':
    main()

Полный вариант смотрите в папке ([вот тут](https://github.com/hse-python-2nd-2019/seminars/tree/master/vk_bot))

## Запускаем и тестируем!